In [12]:
from demoparser import DemoParser
import pandas as pd

parser = DemoParser(r"C:\Program Files (x86)\Steam\steamapps\common\Counter-Strike Global Offensive\csgo\replays\match730_003617425643731419460_0501383489_183.dem")
events = parser.parse_events("smokegrenade_detonate")
events = pd.DataFrame(events)

In [13]:
events = events.drop(['entityid', 'event_name'], axis=1)
events2 = events.drop(['player_steamid','tick','player_name','round'], axis=1)
events2 = events2.values.flatten()

In [14]:
pd.set_option('display.max_rows', 166)
display(events)
pd.reset_option('display.max_rows')

,player_steamid,tick,player_name,z,y,x,round
0,76561199052145085,13049,Jarhad is back,11778.031250,-277.405426,-1608.687012,0
1,76561198038554860,14177,Sir DUCK,11778.031250,645.613831,-1981.732300,0
2,76561198409339370,14315,S1plyi,11745.031250,749.245544,-2204.270264,0
3,76561198023615260,14741,Heinmci,11554.007812,-10.839994,-2132.534424,0
4,76561198048657938,15725,$0$K@ nerealk@,11745.031250,665.449097,-2188.823730,0
5,76561198038554860,19221,Sir DUCK,11778.031250,586.498291,-1980.428223,0
6,76561199052145085,24946,Jarhad is back,11778.031250,-668.069031,-1000.827698,0
7,76561199052145085,28931,Jarhad is back,11778.031250,226.345322,-1398.108521,0
8,76561198048657938,29611,$0$K@ nerealk@,11778.031250,320.440674,-898.210938,0
9,76561199052145085,34595,Jarhad is back,11746.031250,247.133118,-2381.892090,0


In [15]:
a = parser.parse_players()
names = [entry['name'] for entry in a]
print(names)
country_map = {country: index for index, country in enumerate(names)}
country_map

['Sehnsucht', 'S1plyi', 'Domberman', 'Heinmci', 'Jarhad is back', '$0$K@ nerealk@', 'geronimo', 'jadasek', 'Sir DUCK', 'sancheust ♂RIGHT VERSION♂']


{'Sehnsucht': 0,
 'S1plyi': 1,
 'Domberman': 2,
 'Heinmci': 3,
 'Jarhad is back': 4,
 '$0$K@ nerealk@': 5,
 'geronimo': 6,
 'jadasek': 7,
 'Sir DUCK': 8,
 'sancheust ♂RIGHT VERSION♂': 9}

In [22]:
events.to_excel('smokes222.xlsx')

In [41]:
aaa

,X0,Y0,Z0,health0,spotted0,eq_val0,kills0,ammo0,fd0,name0,...,X9,Y9,Z9,health9,spotted9,eq_val9,kills9,ammo9,fd9,name9
0,-1882.659058,624.970520,11776.031250,100,0,2650,5.0,-1.0,0.0,5,...,-1596.320801,-701.809875,11488.03125,100,0,5200,16.0,-1.0,0.000000,2
5,-1915.908203,619.248230,11776.031250,100,0,2650,5.0,8.0,0.0,5,...,-1581.934692,-665.493347,11488.03125,100,0,5200,16.0,-1.0,0.000000,2
10,-1953.323608,613.786377,11776.031250,100,0,2650,5.0,8.0,0.0,5,...,-1567.958252,-629.016907,11488.03125,100,0,5200,16.0,-1.0,0.000000,2
15,-1983.990356,609.701599,11776.031250,100,0,2650,5.0,8.0,0.0,5,...,-1554.157837,-592.473572,11488.03125,100,0,5200,16.0,-1.0,0.000000,2
20,-2022.246582,604.772339,11776.031250,100,0,2650,5.0,-1.0,0.0,5,...,-1540.432739,-555.901672,11488.03125,100,0,5200,16.0,-1.0,0.000000,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
955,-2006.950073,-117.645729,11552.031250,0,0,2650,5.0,NaN,0.0,5,...,-2219.366699,886.871277,11744.03125,0,0,5200,16.0,NaN,3.778606,2
960,-2021.160645,-92.946342,11552.031250,0,0,2650,5.0,NaN,0.0,5,...,-2219.366699,886.871277,11744.03125,0,0,5200,16.0,NaN,3.778606,2
965,-2041.922974,-71.198700,11552.031250,0,0,2650,5.0,NaN,0.0,5,...,-2219.366699,886.871277,11744.03125,0,0,5200,16.0,NaN,3.778606,2
970,-2067.246094,-57.484661,11552.031250,0,0,2650,5.0,NaN,0.0,5,...,-2219.366699,886.871277,11744.03125,0,0,5200,16.0,NaN,3.778606,2


In [5]:
categorical_columns = aaa.select_dtypes(include='object').columns
for column in categorical_columns:
    aaa[column] = aaa[column].map(country_map)

aaa.head(5)

,X0,Y0,Z0,health0,spotted0,eq_val0,kills0,ammo0,fd0,name0,...,X9,Y9,Z9,health9,spotted9,eq_val9,kills9,ammo9,fd9,name9
0,-1603.209473,267.630646,11776.03125,100,0,3500,14.0,-1.0,0.0,9,...,-1531.745361,-1429.242065,11488.03125,100,0,2900,13.0,10.0,0.0,8
5,-1635.675903,247.354889,11776.03125,100,0,3500,14.0,-1.0,0.0,9,...,-1565.722656,-1418.092285,11488.03125,100,0,2900,13.0,-1.0,0.0,8
10,-1667.501221,226.081131,11776.03125,100,0,3500,14.0,-1.0,0.0,9,...,-1602.784790,-1407.603760,11488.03125,100,0,2900,13.0,-1.0,0.0,8
15,-1700.027100,207.571640,11776.03125,100,0,3500,14.0,-1.0,0.0,9,...,-1634.613037,-1388.398560,11488.03125,100,0,2900,13.0,-1.0,0.0,8
20,-1733.958740,198.497421,11776.03125,100,0,3500,14.0,5.0,0.0,9,...,-1660.318726,-1359.890625,11488.03125,100,0,2900,13.0,-1.0,0.0,8


In [6]:
X_train = aaa
y_train = 3

In [30]:
X_train = tf.data.Dataset.from_tensor_slices(dict(aaa))

In [7]:
X_train = np.array([X_train])
X_train

array([[[-1.60320947e+03,  2.67630646e+02,  1.17760312e+04, ...,
          1.00000000e+01,  0.00000000e+00,  8.00000000e+00],
        [-1.63567590e+03,  2.47354889e+02,  1.17760312e+04, ...,
         -1.00000000e+00,  0.00000000e+00,  8.00000000e+00],
        [-1.66750122e+03,  2.26081131e+02,  1.17760312e+04, ...,
         -1.00000000e+00,  0.00000000e+00,  8.00000000e+00],
        ...,
        [-1.44282275e+03,  4.07955475e+02,  1.18450312e+04, ...,
          8.00000000e+00,  0.00000000e+00,  8.00000000e+00],
        [-1.43993335e+03,  4.09903961e+02,  1.18450312e+04, ...,
          8.00000000e+00,  0.00000000e+00,  8.00000000e+00],
        [-1.42870068e+03,  4.24083252e+02,  1.18450312e+04, ...,
         -1.00000000e+00,  0.00000000e+00,  8.00000000e+00]]])

In [8]:
y_train = [y_train]  # Umieszczenie pojedynczej wartości w liście
y_train = np.array([y_train])  # Umieszczenie pojedynczej wartości w tablicy NumPy


In [4]:
X_train.shape

NameError: name 'X_train' is not defined

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM


# Tworzenie modelu
model = Sequential()

# Dodanie warstwy LSTM z 64 jednostkami i wejściowym wymiarem 10
model.add(LSTM(128, input_shape=(1, 196, 100)))
model.add(Dense(5))


model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32)



# Wyświetlenie podsumowania modelu
model.summary()


In [16]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tqdm.auto import tqdm


ticks = events['tick'].values
wanted_props = ["X", "Y", "Z", "health", 'spotted','equipment_value','kills','ammo','flash_duration']
df = parser.parse_ticks(wanted_props)
df = pd.DataFrame(df)

smokes_new = []
indexing = 0
total = len(ticks)
pbar = tqdm(total=total)

for i in ticks:
    index_of_tick = df.index[df['tick'] == i][0]
    
    index_of_tick_start = index_of_tick - 2600
    index_of_tick_end = index_of_tick + 7199
    new_df = df.loc[index_of_tick_start:index_of_tick_end] 

    print(index_of_tick_start, index_of_tick_end)
    new_df = new_df.drop(['steamid'], axis=1)
    new_df = new_df.set_index('tick')
    new_df = new_df.groupby(level=0).agg(list).apply(pd.Series)
    aaa =pd.DataFrame()

    for i in range(10):
        aaa[f'X{i}'] = []
        aaa[f'Y{i}'] = []
        aaa[f'Z{i}'] = []
        aaa[f'health{i}'] = []
        aaa[f'spotted{i}'] = []
        aaa[f'eq_val{i}'] = []
        aaa[f'kills{i}'] = []
        aaa[f'ammo{i}'] = []
        aaa[f'fd{i}'] = []
        aaa[f'name{i}'] = []

    lista = []
    for j in range(len(new_df['X'].index.unique())):
        for i in range(10):
            lista.append(new_df['X'].values[j][i])
            lista.append(new_df['Y'].values[j][i])
            lista.append(new_df['Z'].values[j][i])
            lista.append(new_df['health'].values[j][i])
            lista.append(new_df['spotted'].values[j][i])
            lista.append(new_df['equipment_value'].values[j][i])
            lista.append(new_df['kills'].values[j][i])
            lista.append(new_df['ammo'].values[j][i])
            lista.append(new_df['flash_duration'].values[j][i])
            lista.append(new_df['name'].values[j][i])
        aaa.loc[len(aaa)] = lista
        lista = []
    aaa = aaa[aaa.index % 5 == 0]
    combined_row = aaa.values.flatten()
    combined_row = np.append(combined_row, events2[indexing:indexing+3])
    smokes_new.append(combined_row)

    indexing += 3
    pbar.update(1)


  0%|          | 0/57 [00:00<?, ?it/s]

58685 68484
64325 74124
65015 74814
67145 76944
72065 81864
89535 99334
118145 127944
138065 147864
141465 151264
166375 176174
167645 177444
195115 204914
199855 209654
212565 222364
215215 225014
217645 227444
218585 228384
219485 229284
223315 233114
223965 233764
242515 252314
242715 252514
242855 252654
244995 254794
253565 263364
274405 284204
274455 284254
277475 287274
277735 287534
282155 291954
288615 298414
310915 320714
311485 321284
312105 321904
342735 352534
343145 352944
353995 363794
366425 376224
369195 378994
400065 409864
408505 418304
414635 424434
415355 425154
429545 439344
431335 441134
449245 459044
449345 459144
463865 473664
468095 477894
476045 485844
476365 486164
493695 503494
512835 522634
518285 528084
518755 528554
526155 535954
526905 536704


In [ ]:
import numpy as np
from sklearn.preprocessing import LabelEncoder


ticks = events['tick'].values
wanted_props = ["X", "Y", "Z", "health", 'spotted','equipment_value','kills','ammo','flash_duration']
df = parser.parse_ticks(wanted_props)
df = pd.DataFrame(df)

smokes_new = []
indexing = 0
for i in ticks:
    index_of_tick = df.index[df['tick'] == i][0]
    
    index_of_tick_start = index_of_tick - 2600
    index_of_tick_end = index_of_tick + 7199
    new_df = df.loc[index_of_tick_start:index_of_tick_end] 

    print(index_of_tick_start, index_of_tick_end)
    new_df = new_df.drop(['steamid'], axis=1)
    combined_row = new_df.values.flatten()
    combined_row = np.append(combined_row, events2[indexing:indexing+3])
    smokes_new.append(combined_row)

    indexing += 3


In [17]:
smokes_new = pd.DataFrame(smokes_new)
smokes_new['thrower_name'] = events['player_name']
smokes_new = smokes_new.fillna(-2)
smokes_new.head(5)

,0,1,2,3,4,5,6,7,8,9,...,19594,19595,19596,19597,19598,19599,19600,19601,19602,thrower_name
0,-1467.469971,-1428.660034,11488.031250,100,0,5100,4.0,-1.0,0.0,Sir DUCK,...,0,1900,-2.0,7.0,0.000000,Jarhad is back,11778.031250,-277.405426,-1608.687012,Jarhad is back
1,-2132.003418,-74.950195,11573.009766,100,0,5100,4.0,30.0,0.0,Sir DUCK,...,0,1900,-2.0,-2.0,0.000000,Jarhad is back,11778.031250,645.613831,-1981.732300,Sir DUCK
2,-2251.445801,-261.529236,11617.757812,100,0,5100,4.0,30.0,0.0,Sir DUCK,...,1,1900,-2.0,-2.0,0.000000,Jarhad is back,11745.031250,749.245544,-2204.270264,S1plyi
3,-2617.392822,577.216614,11753.042969,100,0,5100,4.0,30.0,0.0,Sir DUCK,...,0,1900,-2.0,-2.0,5.025002,Jarhad is back,11554.007812,-10.839994,-2132.534424,Heinmci
4,-2443.382568,616.774902,11744.031250,0,0,5100,4.0,-2.0,0.0,Sir DUCK,...,0,200,-2.0,12.0,0.000000,Jarhad is back,11745.031250,665.449097,-2188.823730,$0$K@ nerealk@


In [18]:
categorical_columns = smokes_new.select_dtypes(include='object').columns
for column in categorical_columns:
    smokes_new[column] = smokes_new[column].map(country_map)

smokes_new.head(5)

,0,1,2,3,4,5,6,7,8,9,...,19594,19595,19596,19597,19598,19599,19600,19601,19602,thrower_name
0,-1467.469971,-1428.660034,11488.031250,100,0,5100,4.0,-1.0,0.0,8,...,0,1900,-2.0,7.0,0.000000,4,11778.031250,-277.405426,-1608.687012,4
1,-2132.003418,-74.950195,11573.009766,100,0,5100,4.0,30.0,0.0,8,...,0,1900,-2.0,-2.0,0.000000,4,11778.031250,645.613831,-1981.732300,8
2,-2251.445801,-261.529236,11617.757812,100,0,5100,4.0,30.0,0.0,8,...,1,1900,-2.0,-2.0,0.000000,4,11745.031250,749.245544,-2204.270264,1
3,-2617.392822,577.216614,11753.042969,100,0,5100,4.0,30.0,0.0,8,...,0,1900,-2.0,-2.0,5.025002,4,11554.007812,-10.839994,-2132.534424,3
4,-2443.382568,616.774902,11744.031250,0,0,5100,4.0,-2.0,0.0,8,...,0,200,-2.0,12.0,0.000000,4,11745.031250,665.449097,-2188.823730,5


In [16]:

smokes_new['stopped_rush'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
smokes_new['label'] = [3,3,4,4,3,4,3,4,3,4,4,2,2,2,1,2,3,3,3,4,3,4,3,4]
#smokes_new['stopped_rush'] = [0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
#smokes_new['label'] = [3,3,3,4,4,3,5,3,3,4,3,3,4,2,3,3,3,2,3,2,3,4,4,4,2,2,3,3,3,2,3,3,2,3,3,4,3,3,4,3,3,3,3,4,3,3,3,4,2,3,3,4,3,3,4,4,3,3]
#smokes_new['stopped_rush'] = [0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
#smokes_new['label'] = [3,2,5,4,3,4,3,3,4,3,3,3,3,3,3,4,3,3,3,2,4,3,4,3,3,4,4,4,4,3,3,3,3,3,3,3,3,3,3,4,3,4,2,4,3,4,3,3,3,5,2,2,4,3,3,3,2,3,3,3,3,3,3,4,3,4,3,3,3,4,4,3,3,3,4,4,3,3,3,3,3,4,4,3]
#smokes_new['stopped_rush'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
#smokes_new['label'] = [3,3,3,3,3,3,4,4,3,4,4,3,4,3,3,4,2,3,5,3,3,3,3,3,3,4,3,3,3,4,4,3,4,3,4,4,3,3,3,4,3,3,3,4,3,2,3,3,3,3,3,4,2,3,3,4,3,3,3,4,4,3,4,3,4,2,3,4,3,3,2,2,3,3,3,3,2,2,2,3,3,3,4,3,4,2,3,3,3,4,3,4,3,3,2,3,4,4,3,3,3,4,3,2]
smokes_new.head(5)

C:\Users\tymot\AppData\Local\Temp\ipykernel_28936\3022691362.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  smokes_new['stopped_rush'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
C:\Users\tymot\AppData\Local\Temp\ipykernel_28936\3022691362.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  smokes_new['label'] = [3,3,4,4,3,4,3,4,3,4,4,2,2,2,1,2,3,3,3,4,3,4,3,4]


,0,1,2,3,4,5,6,7,8,9,...,19596,19597,19598,19599,19600,19601,19602,thrower_name,stopped_rush,label
0,-716.682373,-469.234436,11546.420898,100,0,850,16.0,-1.0,0.0,9,...,3.0,-2.0,0.0,2,11778.03125,-478.792023,-421.387024,1,0,3
1,-1991.968750,-383.060181,11552.031250,100,0,1700,3.0,7.0,0.0,9,...,3.0,20.0,0.0,2,11605.84375,-152.058777,-2395.410400,8,0,3
2,-1791.196289,-1022.862671,11506.209961,100,0,4900,4.0,-1.0,0.0,9,...,5.0,30.0,0.0,2,11778.03125,-1147.449097,-1098.369995,6,0,4
3,-752.021545,-299.436920,11488.031250,68,0,4700,6.0,11.0,0.0,9,...,6.0,13.0,0.0,2,11778.03125,-494.484528,-297.720703,9,0,4
4,-347.086029,-683.040710,11774.160156,68,0,4700,6.0,-1.0,0.0,9,...,6.0,20.0,0.0,2,11778.03125,-1058.366455,-1088.367554,6,0,3


In [151]:
#smokes.to_csv('smokes_data.csv',header='False', index='False', mode='a')

In [17]:
import csv

data = smokes_new.values.tolist()
with open('smokes_data.csv', 'a', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(data)

In [1]:
import pandas as pd
smokes = pd.read_csv('smokes_data.csv', header=None)
#smokes = smokes.drop(0)
#smokes = smokes.drop('Unnamed: 0', axis=1)
smokes

,0,1,2,3,4,5,6,7,8,9,...,19596,19597,19598,19599,19600,19601,19602,19603,19604,19605
0,-1588.800903,-1251.655762,11488.03125,100.0,0.0,850.0,2.0,-1.0,0.0,2.0,...,-2.0,-2.0,0.0,3.0,11746.031250,244.179291,-2386.478760,8.0,0.0,3.0
1,-2025.269409,-205.869781,11552.03125,100.0,0.0,3300.0,1.0,35.0,0.0,2.0,...,1.0,21.0,0.0,3.0,11605.843750,-144.030716,-2393.660400,2.0,0.0,3.0
2,-2007.099487,-44.599091,11552.03125,100.0,0.0,3300.0,1.0,35.0,0.0,2.0,...,1.0,21.0,0.0,3.0,11778.031250,-492.871948,-250.106354,9.0,0.0,3.0
3,-2373.782227,-107.928177,11552.03125,100.0,0.0,3300.0,2.0,-1.0,0.0,2.0,...,2.0,-1.0,0.0,3.0,11778.031250,-341.490021,-436.412476,3.0,0.0,3.0
4,-928.019897,-400.103546,11488.03125,0.0,0.0,4200.0,3.0,-2.0,0.0,2.0,...,3.0,-1.0,0.0,3.0,11778.031250,-637.747803,-1497.469849,0.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,-1031.319946,861.966980,11776.03125,100.0,0.0,1900.0,14.0,-1.0,0.0,9.0,...,15.0,30.0,0.0,2.0,11778.031250,2.545478,-1949.236450,1.0,0.0,4.0
266,-1788.485962,83.591194,11776.03125,100.0,0.0,1900.0,14.0,7.0,0.0,9.0,...,15.0,28.0,0.0,2.0,11605.843750,-134.834305,-2376.312988,4.0,0.0,3.0
267,-1469.604980,-303.569519,11776.03125,100.0,1.0,1900.0,14.0,6.0,0.0,9.0,...,15.0,28.0,0.0,2.0,11778.031250,-820.171326,-1167.763672,8.0,0.0,4.0
268,-944.854004,858.479004,11776.03125,100.0,0.0,1450.0,14.0,8.0,0.0,9.0,...,16.0,-2.0,0.0,2.0,11778.016602,15.947900,-1976.875000,1.0,0.0,3.0


In [2]:
from sklearn.model_selection import train_test_split
features = smokes.iloc[:, :-2]
labels = smokes[19605]


train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.001, random_state=35)


In [ ]:
pd.set_option('display.max_rows', 270)
display(smokes)
pd.reset_option('display.max_rows')

In [16]:
survived = smokes[smokes[64014]==1]
deceased = smokes[smokes[64014]==0]
deceased = deceased.sample(n=len(survived), random_state=101)
dfy = pd.concat([survived,deceased],axis=0)

In [3]:
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error

label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

model = xgb.XGBClassifier()
model.fit(train_features, train_labels_encoded) #xgb_model=r"C:\Users\tymot\Desktop\Projekty\Prawdopodobieństwa fragów\xgboostModel")

pred_labels_encoded = model.predict(test_features)
pred_labels = label_encoder.inverse_transform(pred_labels_encoded)

accuracy = accuracy_score(test_labels, pred_labels)
print('accuracy:',accuracy)
accuracy = mean_absolute_error(test_labels, pred_labels)
print('mse:',accuracy)


accuracy: 0.0
mse: 2.0


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, mean_absolute_error

# Zamień 1, 2, 3, 4, 5 na 0, 1, 2, 3, 4
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Konwertuj dane wejściowe na numpy array
train_features_array = train_features.values.astype(np.float32)
train_labels_array = train_labels_encoded
test_features_array = test_features.values.astype(np.float32)

# Konwertuj dane wejściowe na tensory PyTorch
train_features_tensor = torch.from_numpy(train_features_array)
train_labels_tensor = torch.from_numpy(train_labels_array)
test_features_tensor = torch.from_numpy(test_features_array)

# Tworzenie modelu w PyTorch
model = nn.Sequential(
    nn.Linear(19604, 64),
    nn.Softmax(dim=1),
    nn.Linear(64, 32),
    nn.Softmax(dim=1),
    nn.Linear(32, 5),
    nn.Softmax(dim=1)
)

# Definiowanie funkcji straty i optymalizatora
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Trenowanie modelu
for epoch in range(500):
    optimizer.zero_grad()
    outputs = model(train_features_tensor)
    loss = criterion(outputs, train_labels_tensor)
    loss.backward()
    optimizer.step()

# Predykcja etykiet dla danych testowych
model.eval()
with torch.no_grad():
    pred_labels_encoded = model(test_features_tensor)
    _, pred_labels_encoded = torch.max(pred_labels_encoded, 1)
    pred_labels = label_encoder.inverse_transform(pred_labels_encoded.numpy())

# Obliczanie dokładności i MAE
accuracy = accuracy_score(test_labels, pred_labels)
print('accuracy:', accuracy)

mae = mean_absolute_error(test_labels, pred_labels)
print('MAE:', mae)


accuracy: 0.16666666666666666
MAE: 0.9814814814814815


In [7]:
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error

#Zamień 1,2,3,4,5 na 0,1,2,3,4 
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='softmax', input_shape=(19604,)),
    tf.keras.layers.Dense(64, activation='softmax'),
    tf.keras.layers.Dense(5, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_features, train_labels_encoded, epochs=100)

pred_labels_encoded = model.predict(test_features)
pred_labels_encoded = tf.argmax(pred_labels_encoded, axis=1)
pred_labels = label_encoder.inverse_transform(pred_labels_encoded)

accuracy = accuracy_score(test_labels, pred_labels)
print('accuracy:', accuracy)

mae = mean_absolute_error(test_labels, pred_labels)
print('MAE:', mae)


Epoch 1/100
7/7 [==============================] - 0s 16ms/step - loss: 1.6019 - accuracy: 0.0556
Epoch 2/100
7/7 [==============================] - 0s 15ms/step - loss: 1.5880 - accuracy: 0.5741
Epoch 3/100
7/7 [==============================] - 0s 10ms/step - loss: 1.5744 - accuracy: 0.5741
Epoch 4/100
7/7 [==============================] - 0s 3ms/step - loss: 1.5616 - accuracy: 0.5741
Epoch 5/100
7/7 [==============================] - 0s 3ms/step - loss: 1.5485 - accuracy: 0.5741
Epoch 6/100
7/7 [==============================] - 0s 3ms/step - loss: 1.5359 - accuracy: 0.5741
Epoch 7/100
7/7 [==============================] - 0s 3ms/step - loss: 1.5236 - accuracy: 0.5741
Epoch 8/100
7/7 [==============================] - 0s 3ms/step - loss: 1.5113 - accuracy: 0.5741
Epoch 9/100
7/7 [==============================] - 0s 3ms/step - loss: 1.4996 - accuracy: 0.5741
Epoch 10/100
7/7 [==============================] - 0s 3ms/step - loss: 1.4882 - accuracy: 0.5741
Epoch 11/100
7/7 [========

In [38]:
smokes.iloc[84]

0        -1641.496094
1          262.317566
2        11776.031250
3          100.000000
4            0.000000
             ...     
19601     -148.699356
19602    -2407.555176
19603        1.000000
19604        0.000000
19605        4.000000
Name: 84, Length: 19606, dtype: float64

In [4]:
#new_data = smokes_new.iloc[35]
new_data = smokes.iloc[26].drop([19605,19604])
new_data = pd.DataFrame(new_data.values)
new_data = new_data.T

new_data = new_data.values.astype(np.float32)
new_features_tensor = torch.from_numpy(new_data.astype(np.float32))

model.eval()

# Predykcja etykiet dla nowych danych wejściowych
# Predykcja etykiet dla nowych danych wejściowych
with torch.no_grad():
    pred_probs = model(new_features_tensor)
    pred_probs = torch.softmax(pred_probs, dim=1)

# Wyświetlenie prawdopodobieństw dla wszystkich klas
class_probabilities = pred_probs.numpy()
for i, probabilities in enumerate(class_probabilities):
    class_label = label_encoder.inverse_transform([i])[0]
    print(f"Class: {class_label}, Probabilities: {probabilities}")



#model.predict(new_data)


Class: 1.0, Probabilities: [0.2033956  0.19527237 0.19744669 0.20598622 0.19789909]


In [43]:
new_data

,0,1,2,3,4,5,6,7,8,9,...,19594,19595,19596,19597,19598,19599,19600,19601,19602,19603
0,-1641.496094,262.317566,11776.03125,100.0,0.0,2050.0,21.0,-1.0,0.0,2.0,...,0.0,4550.0,12.0,-1.0,0.0,3.0,11605.84375,-148.699356,-2407.555176,1.0


In [33]:
smokes[19605].value_counts()

3.0    161
4.0     74
2.0     30
5.0      4
1.0      1
Name: 19605, dtype: int64

In [4]:
#new_data = smokes.iloc[160].drop([19605,19604])
new_data = smokes_new.iloc[31]
new_data = pd.DataFrame(new_data.values)
new_data = new_data.T
model.predict_proba(new_data)


array([[1.175473e-03, 9.850219e-01, 7.666926e-03, 5.182455e-03,
        9.533052e-04]], dtype=float32)

In [5]:
new_data = smokes.iloc[51].drop([19605,19604])
#new_data = smokes_new.iloc[31]
new_data = pd.DataFrame(new_data.values)
new_data = new_data.T
model.predict(new_data)


1/1 [==============================] - 0s 17ms/step


array([[0.3345123 , 0.5238128 , 0.46901295, 0.64262164, 0.2153    ]],
      dtype=float32)

In [19]:
#new_data = smokes.iloc[84].drop([107805,107806])
for i in range(len(smokes_new)):
    new_data = smokes_new.iloc[i]
    new_data = pd.DataFrame(new_data.values)
    new_data = new_data.T
    temp = model.predict_proba(new_data)
    temp = temp[0]

    suma = 0
    iter = 1
    for j in temp:
        suma += j * iter
        iter += 1
    print(i, suma, temp)


0 3.5156821929849684 [0.00400811 0.02422454 0.42652145 0.5425687  0.00267718]
1 3.0105853066779673 [0.0036649  0.0102771  0.96015936 0.02360484 0.00229375]
2 3.0356222172267735 [0.00966605 0.01601866 0.909775   0.05810757 0.00643271]
3 3.0590865472331643 [0.00583514 0.02168173 0.8837492  0.08502937 0.00370458]
4 3.958829419570975 [0.0016652  0.00499324 0.02732269 0.9648847  0.00113417]
5 3.24394541233778 [0.01159646 0.05553154 0.61669546 0.3096831  0.00649341]
6 3.35600695759058 [0.01155142 0.01328669 0.592774   0.37237957 0.01000839]
7 3.6716042943298817 [0.00920689 0.05465649 0.22542714 0.6767443  0.03396518]
8 3.3300618585199118 [0.02613519 0.05047688 0.5429245  0.32811788 0.05234557]
9 3.1684921756386757 [0.01103021 0.03173129 0.7570534  0.1780863  0.02209882]
10 3.3475708812475204 [0.01778246 0.04437606 0.5429065  0.36235806 0.0325769 ]
11 3.676791832782328 [0.00213423 0.00346834 0.31208533 0.68009573 0.0022164 ]
12 3.1718804771080613 [0.00348572 0.0166547  0.79355115 0.17711027 0

In [9]:
new_data

,0,1,2,3,4,5,6,7,8,9,...,64004,64005,64006,64007,64008,64009,64010,64011,64012,64013
0,-971.916504,757.763428,11776.03125,100.0,0.0,5800.0,4400.0,6.0,-1.0,0.0,...,0.0,5800.0,4400.0,6.0,-2.0,0.0,11531.902344,-549.474792,-772.823547,0.0


In [63]:
smokes.value_counts([107805])

107805
3.0       161
4.0        74
2.0        30
5.0         4
1.0         1
dtype: int64

In [13]:
model.save_model("xgboostModel")

In [ ]:
pd.set_option('display.max_rows', 277)
display(smokes)
pd.reset_option('display.max_rows')